<a href="https://colab.research.google.com/github/FrancescoRito/Dashboard_F1/blob/main/table_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import date
import json as j
import pandas as pd
import requests as r

#Find current year
year = date.today().year

#create a list from 1950 to current year
years0=list(range(1950,year))

#create a copy, rename and cast to string of list of the years
base_url=years0.copy()
for i in range(len(base_url)):
    base_url[i] = 'http://ergast.com/api/f1/'+str(base_url[i])+'/results.json?limit=1000'

#Create lists for the final dataframe
seasons = []
rounds = []
raceNames = []
circuits = []
drivers = []
positions = []
points = []
status = []
constructors = []
laps = []
bestLaps = []

#Looping to GET the data and transform them in json
for y in range(len(base_url)):
    response = r.get(base_url[y])
    data = response.json()

#Looping through the json file and fill the lists
    for race in data['MRData']['RaceTable']['Races']:
      for result in race['Results']:
        seasons.append(data['MRData']['RaceTable']['season'])
        rounds.append(race['round'])
        raceNames.append(race['raceName'])
        circuits.append(race['Circuit']['circuitId'])
        drivers.append(result['Driver']['driverId'])
        positions.append(result['position'])
        points.append(result['points'])
        status.append(result['status'])
        constructors.append(result['Constructor']['constructorId'])
        laps.append(result['laps'])
        if 'FastestLap' in result:
          bestLaps.append(str(result['FastestLap']['Time']['time']))
        else:
          bestLaps.append(' ')


#Create the dataframe
results_df = pd.DataFrame({'season': seasons,
                          'round': rounds,
                          'raceName': raceNames,
                          'circuit': circuits,
                          'driver': drivers,
                          'position': positions,
                          'points': points,
                          'status': status,
                          'constructor': constructors,
                          'laps': laps,
                          'FastestLap' : bestLaps
                         })
#create the column id.race
results_df['id.race'] = results_df.apply(lambda row: 'id.race_' + str(row['season']) + '-' + str(row['round']), axis=1)

# Define the function for the column type
def race_type(row):
    if (row['season'] == '2021' and row['round'] in [10, 14, 19]) or (row['season'] == '2022' and row['round'] in [4, 11, 21]):
        return 'sprint'
    else:
        return 'classic'

# Create the new column "type" and apply the function to each row
results_df['type'] = results_df.apply(race_type, axis=1)


#moving the new columns
cols = results_df.columns.tolist()
cols.insert(0, cols.pop(cols.index('id.race')))
cols.insert(3, cols.pop(cols.index('type')))
results_df = results_df.reindex(columns= cols)

results_df.to_csv('results_df_3.csv')
